https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata

In [4]:
import requests
import sys
import os
import pandas as pd
import time
import logging
import shutil

In [5]:
# Zona horaria de Perú
os.environ['TZ'] = 'America/Lima'
time.tzset()  # Solo funciona en sistemas tipo Unix (como Colab o Linux)

In [6]:
# Reset y configurar logging para que funcione en Colab / Jupyter
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

In [ ]:
# Constantes
GITHUB_ROOT_PATH = 'https://github.com/marcomobar/datasets/raw/refs/heads/main'
# ROOT_DIR = '/content/drive/MyDrive/Datasets/datalake'
ROOT_DIR = ''
OUTPUT_DIR = os.path.join(ROOT_DIR, 'datalake_outputs')

In [8]:
def get_config_file(file_name: str) -> dict[str:str]:
    """
    Obtiene las rutas del archivo de configuración.
    
    Parámetros:
    - file_name (str): Nombre del archivo a obtener.
    
    Retorna:
    - dict[str,str]: Diccionario con claves "GITHUB_FILE", "GITHUB_FILE_PATH" y "OUTPUT_FILE_PATH".
    """
    GITHUB_FILE_PATH = os.path.join(GITHUB_ROOT_PATH, file_name)  #(1) Construye la ruta completa del archivo en el repositorio de GitHub.
    OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, file_name)  #(2) Construye la ruta local de salida donde se guardará el archivo.

    return {
        'GITHUB_FILE': file_name,
        'GITHUB_FILE_PATH': GITHUB_FILE_PATH,
        'OUTPUT_FILE_PATH': OUTPUT_FILE_PATH
    }

In [9]:
def validar_existencia_archivo(path: str) -> bool:
    """
    Verifica si existe el archivo en la ruta remota.
    
    Parámetros:
    - path (str): URL o ruta del archivo a validar.
    
    Retorna:
    - bool: True si el archivo existe (status 200), False en caso contrario.
    """
    response = requests.head(path, allow_redirects=True)  #(3) Realiza una petición HEAD a la URL para verificar existencia del recurso.
    if response.status_code != 200:
        logging.warning(f"El archivo no existe en el repositorio: {path}")  #(4) Registra una advertencia indicando que el archivo no existe en el repositorio.
        return False
    return True

In [10]:
def limpiar_directorio(path: str) -> None:
    """
    Elimina todos los archivos y subdirectorios de un directorio. Si no existe, lo crea.
    
    Parámetros:
    - path (str): Ruta del directorio a limpiar.
    
    Retorna:
    - None: No retorna valor.
    """
    if not os.path.exists(path):  #(5) Comprueba si el directorio de salida no existe.
        os.makedirs(path)  #(6) Crea el directorio, incluyendo carpetas intermedias.
        return

    for item in os.listdir(path):  #(7) Itera sobre cada elemento en el directorio.
        ruta = os.path.join(path, item)  #(8) Construye la ruta completa de cada elemento dentro del directorio.
        if os.path.isfile(ruta):  #(9) Verifica si la ruta corresponde a un archivo regular.
            os.remove(ruta)  #(10) Elimina el archivo encontrado.
        elif os.path.isdir(ruta):  #(11) Verifica si la ruta corresponde a un subdirectorio.
            shutil.rmtree(ruta)  #(12) Elimina el directorio y todo su contenido de forma recursiva.
    logging.info(f"Directorio limpiado: {path}")

In [11]:
def leer_y_guardar_csv_en_datalake(file_path: str, output_path: str) -> None:
    """
    Lee un CSV y lo guarda en la ruta de salida.
    
    Parámetros:
    - file_path (str): Ruta del archivo CSV de entrada.
    - output_path (str): Ruta donde se guardará el CSV resultante.
    
    Retorna:
    - None: No retorna valor.
    """
    try:
        df_csv = pd.read_csv(file_path)  #(13) Lee el CSV en un DataFrame de pandas.
        df_csv.to_csv(output_path, index=False)  #(14) Guarda el DataFrame en CSV sin índices.
        logging.info(f"Guardado exitosamente: {output_path}")
    except Exception as e:  #(15) Captura y registra cualquier excepción ocurrida al procesar el CSV.
        logging.error(f"Error al procesar archivo {file_path}: {e}")
        raise

In [12]:
def procesar_archivo_desde_github(file_name: str = 'dummy.csv') -> None:
    """
    Procesa un archivo desde GitHub: valida existencia, descarga y guarda.
    
    Parámetros:
    - file_name (str): Nombre del archivo CSV en el repositorio GitHub.
    
    Retorna:
    - None: No retorna valor.
    """
    logging.info(f"{'='*20} PROCESANDO: {file_name} {'='*20}")

    config_file = get_config_file(file_name)  #(16) Obtiene configuración de rutas para el nombre de archivo dado.
    GITHUB_FILE_PATH = config_file['GITHUB_FILE_PATH']  #(17) Extrae la ruta de GitHub del diccionario de configuración.
    OUTPUT_FILE_PATH = config_file['OUTPUT_FILE_PATH']  #(18) Extrae la ruta de salida local del diccionario de configuración.

    # Crear directorio si no existe
    os.makedirs(OUTPUT_DIR, exist_ok=True)  #(19) Crea el directorio de salida si no existe.

    if not validar_existencia_archivo(GITHUB_FILE_PATH):
        logging.warning(f"Archivo no encontrado: {GITHUB_FILE_PATH}")
        return

    leer_y_guardar_csv_en_datalake(GITHUB_FILE_PATH, OUTPUT_FILE_PATH)  #(20) Procesa el CSV remoto y lo guarda en local.

    logging.info(f"{'='*20} FINALIZADO: {file_name} {'='*20}")

In [13]:
def main():
    """
    Punto de entrada principal: limpia directorio, pausa y procesa archivos listados.
    
    Retorna:
    - None: No retorna valor.
    """
    limpiar_directorio(OUTPUT_DIR)  #(21) Limpia el directorio de salida antes de iniciar nuevos procesos.
    time.sleep(5)  # Solo para demostración visual, puede eliminarse en producción

    archivos = [
        'Airbnb_Open_Data_25M.csv'
    ]  #(22) Define la lista de archivos CSV a procesar.

    for archivo in archivos:  #(23) Itera sobre cada nombre de archivo en la lista.
        procesar_archivo_desde_github(archivo)  #(24) Llama a la función de procesamiento para cada archivo.

In [14]:
# Sólo se ejecuta si el archivo se ejecuta directamente
if __name__ == "__main__":  #(25) Ejecuta main() al ejecutar el script directamente.
    main()

2025-04-17 15:06:55,026 - INFO - Directorio limpiado: datalake_outputs
2025-04-17 15:07:00,031 - INFO - ==================== PROCESANDO: Airbnb_Open_Data_25M.csv ====================


/var/folders/t_/qntbphvs7tqccq0dmrcml8sm0000gn/T/ipykernel_39111/775702360.py:13: DtypeWarning: Columns (11,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(file_path)  #(13) Lee el CSV en un DataFrame de pandas.


2025-04-17 15:07:11,244 - INFO - Guardado exitosamente: datalake_outputs/Airbnb_Open_Data_25M.csv
2025-04-17 15:07:11,246 - INFO - ==================== FINALIZADO: Airbnb_Open_Data_25M.csv ====================
